In [ ]:
import csv
import os
import sys
import time

import tensorflow as tf
from tensorflow.keras.callbacks import (EarlyStopping, LambdaCallback,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.mixed_precision import Policy, set_global_policy
from tensorflow.keras.models import load_model

policy = Policy('mixed_float16')
set_global_policy(policy)
tf.keras.mixed_precision.set_global_policy('float32')  # Sử dụng float32 cho tất cả

path_to_add = os.path.abspath(os.path.join('..', 'src'))
if path_to_add in sys.path:
    sys.path.remove(path_to_add)
sys.path.append(path_to_add)

from models.fcn import fcn_model
from models.unet import unet_model
from models.segnet import segnet_model
from models.unetpp import unetpp_model
from utils.data_loader import create_dataset
from utils.metrics import dice_loss, iou_metric

In [ ]:
# Thiết lập chế độ eager execution
tf.config.run_functions_eagerly(True)

# Thiết lập tham số
batch_size = 1  # Kích thước batch
images_path = r'.\data\processed\images'
masks_path = r'.\data\processed\mask'
loss = tf.keras.losses.BinaryCrossentropy()

# Nhập số lượng mẫu
num_samples = 4758
generator = create_dataset(images_path, masks_path, batch_size=batch_size, num_samples=num_samples, data_ratio=50).repeat()

# Đường dẫn lưu mô hình và log
model_save_path = r".\experiments\fcn_experiment\lr1e_5.keras"
log_file_path = r".\logs\fcn\lr1e_5.csv"

steps_per_epoch = num_samples // batch_size

if os.path.exists(model_save_path):
    try:
        model = load_model(model_save_path, custom_objects={'loss': dice_loss, 'iou_metric': iou_metric})
        initial_epoch = model.optimizer.iterations.numpy() // steps_per_epoch
        print("Mô hình đã được tải thành công từ:", model_save_path)
    except Exception as e:
        print("Lỗi khi tải mô hình:", e)
        model = fcn_model((512, 512, 1))
        initial_epoch = 0
else:
    model = fcn_model((512, 512, 1))
    initial_epoch = 0

In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.0)
model.compile(optimizer=optimizer, loss=dice_loss, metrics=['accuracy', iou_metric])

# Các callback để theo dõi quá trình huấn luyện
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(model_save_path, monitor='loss', save_best_only=True, mode='min', verbose=2)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

In [ ]:
if not os.path.exists(log_file_path):
    with open(log_file_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "loss", "accuracy", "iou_metric", "learning_rate", "epoch_time"])

# Hàm callback để ghi log sau mỗi epoch
def on_epoch_begin(epoch, logs):
    global epoch_start_time
    epoch_start_time = time.time()  # Ghi lại thời gian bắt đầu của mỗi epoch

def on_epoch_end(epoch, logs):
    epoch_time = time.time() - epoch_start_time  # Tính toán thời gian hoàn thành của epoch
    current_lr = model.optimizer.learning_rate.numpy()
    with open(log_file_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            epoch + 1,
            logs.get("loss", None),
            logs.get("accuracy", None),
            logs.get("iou_metric", None),
            current_lr,
            epoch_time
        ])

log_callback = LambdaCallback(on_epoch_begin=on_epoch_begin, on_epoch_end=on_epoch_end)

# Danh sách callbacks
callbacks = [checkpoint, early_stopping, reduce_lr, log_callback]

# Huấn luyện mô hình
model.fit(
    generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    callbacks=callbacks,
    initial_epoch=initial_epoch
)

In [ ]:
os.system("shutdown /s /t 1")